# Same problem/success as log_ret_many_indicators...

In [1]:
from ue.uexp.dataprocessing.processor_binance import BinanceProcessor
from ue.uexp.dataprocessing.func import *
from ue.uexp.dataprocessing.ta import *
from ue.uexp.models.util import *

#======TA==============
from ta.momentum import *
from ta.trend import *
from ta.volatility import *
from ta.wrapper import *
import ta
#=========================
import tensorflow as tf
from tensorflow.keras import layers
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
plt.rcParams["figure.figsize"] = (20, 10)

import warnings
warnings.filterwarnings('ignore')

# Introduction

We use MLP, multiple variables ([log_ret_0, ..., log_ret_29, volume, spread, log_price_change, ...] to predict -> log_return")

## Predicting on
- log_return of price at next timestep

## Data + preprocessing

In [2]:
### get data (set config params)
ticker_list = ["BTCUSDT"]
start_date = "2022-02-03"
end_date = "2022-02-04"
time_interval = "1s"

p = BinanceProcessor("binance")
df = p.download_data(ticker_list, start_date, end_date, time_interval)

Found 1 symbols
[1/1] - start download daily BTCUSDT aggTrades 

file already exists! ./cache/tick_raw/BTCUSDT-aggTrades-2022-02-02.csv

file already exists! ./cache/tick_raw/BTCUSDT-aggTrades-2022-02-03.csv

file already exists! ./cache/tick_raw/BTCUSDT-aggTrades-2022-02-04.csv


In [3]:
df.drop(["tic"], axis=1, inplace=True)
df.columns = ["Open","High","Low","Close","Volume"]

## Adding TAs

#### Volatility Indicators

In [4]:
#df.head(50)

#### Momentum Indicators
- Rate of change (ROC)
- Relative Strenght Index (RSI)
- Stochastic RSI (sRSI, srsi_k, srsi_d)
- True Strength Index (TSI)

In [5]:
window = 30 # a choice
# df[f"roc_{window}"] = ta.momentum.ROCIndicator(close=df["Close"], window=window).roc()

# TSI Indicator
df["momentum_tsi"] = TSIIndicator(
    close=df["Close"], window_slow=25, window_fast=13, fillna=True
).tsi()/100 #scale 0-1

## Drop OHL, stationize C, and related

In [6]:
def stationize(series):
    return np.log(series / series.shift(1))

In [7]:
df.head()

,Open,High,Low,Close,Volume,momentum_tsi
2022-02-03 00:00:00,36896.37,36907.62,36896.36,36902.96,5.20868,0.000000
2022-02-03 00:00:01,36902.97,36902.97,36902.80,36902.81,0.05825,-1.000000
2022-02-03 00:00:02,36901.10,36901.11,36896.39,36899.19,0.72231,-1.000000
2022-02-03 00:00:03,36899.19,36900.54,36895.53,36900.54,0.40235,-0.871596
2022-02-03 00:00:04,36900.53,36900.54,36895.62,36895.62,0.32702,-0.833150


In [8]:
# store initial price to reconstruct it back from returns
# initialprices = {
#     "Close": df['Close'][0],
#     "volume_vwap": ,
#     "trend_sma_slow",
#     "trend_ema_slow":
# }
# the initial price is at 00:00:00
df['log_ret'] = stationize(df['Close'])
# df['log_volume_vwap_ret'] = stationize(df['volume_vwap'])
# df['log_trend_sma_slow_ret'] = stationize(df['trend_sma_slow'])
# df['log_trend_ema_slow_ret'] = stationize(df['trend_ema_slow'])

In [9]:
df.drop(["Open","High","Low", "Volume"], axis=1, inplace=True)

In [10]:
#df.dropna(inplace=True)

## Make Windowed dataset (using dt_feats1)

In [11]:
HORIZON = 1
WINDOW = 30

In [12]:
# Add windowed columns
for i in range(WINDOW-1): # Shift values for each step in WINDOW_SIZE
  df[f"log_ret+{i+2}"] = df["log_ret"].shift(periods=i+2)
df.head(50)

,Close,momentum_tsi,log_ret,log_ret+2,log_ret+3,log_ret+4,log_ret+5,log_ret+6,log_ret+7,log_ret+8,...,log_ret+21,log_ret+22,log_ret+23,log_ret+24,log_ret+25,log_ret+26,log_ret+27,log_ret+28,log_ret+29,log_ret+30
2022-02-03 00:00:00,36902.96,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-02-03 00:00:01,36902.81,-1.000000,-4.064723e-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-02-03 00:00:02,36899.19,-1.000000,-9.810032e-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-02-03 00:00:03,36900.54,-0.871596,3.658550e-05,-4.064723e-06,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-02-03 00:00:04,36895.62,-0.833150,-1.333403e-04,-9.810032e-05,-4.064723e-06,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-02-03 00:00:05,36900.26,-0.600366,1.257523e-04,3.658550e-05,-9.810032e-05,-4.064723e-06,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-02-03 00:00:06,36898.71,-0.504129,-4.200601e-05,-1.333403e-04,3.658550e-05,-9.810032e-05,-4.064723e-06,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-02-03 00:00:07,36898.71,-0.450333,0.000000e+00,1.257523e-04,-1.333403e-04,3.658550e-05,-9.810032e-05,-4.064723e-06,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-02-03 00:00:08,36898.71,-0.416051,0.000000e+00,-4.200601e-05,1.257523e-04,-1.333403e-04,3.658550e-05,-9.810032e-05,-4.064723e-06,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-02-03 00:00:09,36896.07,-0.416555,-7.154978e-05,0.000000e+00,-4.200601e-05,1.257523e-04,-1.333403e-04,3.658550e-05,-9.810032e-05,-4.064723e-06,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
df.dropna().shape

(172769, 32)

In [14]:
df.shape

(172800, 32)

In [15]:
#drop non-stationized values, keeping them for later reconstruction
X = df.dropna().drop(["Close"], axis=1).astype(np.float32) 

#note, starting at :30second
X.head()

,momentum_tsi,log_ret,log_ret+2,log_ret+3,log_ret+4,log_ret+5,log_ret+6,log_ret+7,log_ret+8,log_ret+9,...,log_ret+21,log_ret+22,log_ret+23,log_ret+24,log_ret+25,log_ret+26,log_ret+27,log_ret+28,log_ret+29,log_ret+30
2022-02-03 00:00:31,-0.326270,0.000000,0.000000,0.000000,0.000000,0.000000,-0.000037,-0.000032,-0.000094,-0.000123,...,0.000184,-0.000072,0.000000,0.000000,-0.000042,0.000126,-0.000133,0.000037,-0.000098,-0.000004
2022-02-03 00:00:32,-0.221396,0.000299,0.000373,0.000000,0.000000,0.000000,0.000000,-0.000037,-0.000032,-0.000094,...,-0.000184,0.000184,-0.000072,0.000000,0.000000,-0.000042,0.000126,-0.000133,0.000037,-0.000098
2022-02-03 00:00:33,-0.144487,-0.000010,0.000000,0.000373,0.000000,0.000000,0.000000,0.000000,-0.000037,-0.000032,...,-0.000259,-0.000184,0.000184,-0.000072,0.000000,0.000000,-0.000042,0.000126,-0.000133,0.000037
2022-02-03 00:00:34,-0.083675,0.000010,0.000299,0.000000,0.000373,0.000000,0.000000,0.000000,0.000000,-0.000037,...,0.000249,-0.000259,-0.000184,0.000184,-0.000072,0.000000,0.000000,-0.000042,0.000126,-0.000133
2022-02-03 00:00:35,-0.031504,0.000027,-0.000010,0.000299,0.000000,0.000373,0.000000,0.000000,0.000000,0.000000,...,-0.000151,0.000249,-0.000259,-0.000184,0.000184,-0.000072,0.000000,0.000000,-0.000042,0.000126


In [16]:
X.shape

(172769, 31)

In [17]:
y = df.dropna().drop(["Close"], axis=1)["log_ret"].astype(np.float32)
y.head()

2022-02-03 00:00:31    0.000000
2022-02-03 00:00:32    0.000299
2022-02-03 00:00:33   -0.000010
2022-02-03 00:00:34    0.000010
2022-02-03 00:00:35    0.000027
Freq: S, Name: log_ret, dtype: float32

In [18]:
y.shape

(172769,)

### Train val test split: (first 60% training, next 20% validation, last 20% for evaluation)

In [19]:
# #train80%, validate 20%-1, test on last datapoint
# split_size_1 = int(len(X) * 0.8)
# X_train, y_train = X[:split_size_1], y[:split_size_1]
# X_val, y_val = X[split_size_1:-1], y[split_size_1:-1]
# X_test, y_test = X[-1:], y[-1:]
# len(X_train), len(y_train), len(X_val), len(y_val), len(X_test), len(y_test)

In [20]:
split_size_1 = int(len(X) * 0.6)
split_size_2 = int(len(X) * 0.8)
X_train, y_train = X[:split_size_1], y[:split_size_1]
X_val, y_val = X[split_size_1:split_size_2], y[split_size_1:split_size_2]
X_test, y_test = X[split_size_2:], y[split_size_2:]
len(X_train), len(y_train), len(X_val), len(y_val), len(X_test), len(y_test)

(103661, 103661, 34554, 34554, 34554, 34554)

## Model building and fitting

In [21]:
tf.keras.backend.clear_session()

In [22]:
# Make multivariate time series model
model_dense_multivariate = tf.keras.Sequential([
  layers.Dense(128, activation="relu"),
  layers.Dense(128, activation="relu"), # adding an extra layer here should lead to beating the naive model
  layers.Dense(HORIZON)
], name="model_dense_multivariate")

# Compile
model_dense_multivariate.compile(loss="mae",
                                 optimizer=tf.keras.optimizers.Adam(),
                                 metrics=['mse','mape','mae'])

Metal device set to: Apple M1 Pro


2022-02-10 16:43:27.602395: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-02-10 16:43:27.602520: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [23]:
# Fit
# early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
#                                                     patience=2,
#                                                     mode='min')

history = model_dense_multivariate.fit(X_train, y_train,
                                       epochs=50, #saw that 20 was good enough
                                       batch_size=32,
                                       shuffle=False,
                                       verbose=1, # progress bar
                                       validation_data=(X_val, y_val),
                                       callbacks=[create_model_checkpoint(model_name=model_dense_multivariate.name),
                                                 ])

Epoch 1/50


2022-02-10 16:43:27.656515: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-02-10 16:43:27.809870: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


3240/3240 [==============================] - ETA: 0s - loss: 6.8055e-04 - mse: 6.9925e-06 - mape: 196370.5938 - mae: 6.8055e-04

2022-02-10 16:43:47.335679: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


INFO:tensorflow:Assets written to: model_experiments/model_dense_multivariate/assets
3240/3240 [==============================] - 23s 7ms/step - loss: 6.8055e-04 - mse: 6.9925e-06 - mape: 196370.5938 - mae: 6.8055e-04 - val_loss: 7.6956e-05 - val_mse: 1.3479e-08 - val_mape: 16117.9229 - val_mae: 7.6956e-05
Epoch 2/50
   1/3240 [..............................] - ETA: 1:22 - loss: 1.5354e-04 - mse: 3.9870e-08 - mape: 7310.8398 - mae: 1.5354e-04

2022-02-10 16:43:50.549171: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


3240/3240 [==============================] - 22s 7ms/step - loss: 1.1133e-04 - mse: 2.1985e-08 - mape: 29083.5684 - mae: 1.1133e-04 - val_loss: 9.9460e-05 - val_mse: 1.7411e-08 - val_mape: 27018.6523 - val_mae: 9.9460e-05
Epoch 3/50
3240/3240 [==============================] - 22s 7ms/step - loss: 1.1731e-04 - mse: 3.3528e-08 - mape: 31480.6250 - mae: 1.1731e-04 - val_loss: 8.4694e-05 - val_mse: 1.4676e-08 - val_mape: 20070.1270 - val_mae: 8.4694e-05
Epoch 4/50
3240/3240 [==============================] - 21s 7ms/step - loss: 1.1412e-04 - mse: 2.2985e-08 - mape: 29901.3535 - mae: 1.1412e-04 - val_loss: 1.0309e-04 - val_mse: 1.7647e-08 - val_mape: 27749.0977 - val_mae: 1.0309e-04
Epoch 5/50
3240/3240 [==============================] - 22s 7ms/step - loss: 1.1118e-04 - mse: 2.1700e-08 - mape: 28821.9883 - mae: 1.1118e-04 - val_loss: 2.2667e-04 - val_mse: 6.0375e-08 - val_mape: 76702.1641 - val_mae: 2.2667e-04
Epoch 6/50
2829/3240 [=========================>....] - ETA: 2s - loss: 1.1263e

KeyboardInterrupt: 

In [ ]:
model_dense_multivariate.summary()

In [ ]:
import pydot

In [ ]:
from keras.utils.vis_utils import plot_model
plot_model(model_dense_multivariate)

## Examining Histories

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
print(history.history.keys())

In [ ]:
plt.plot(history.history['mse'])
plt.plot(history.history['val_mse'])
plt.title('model mean squared error')
plt.ylabel('mean squared error')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper right')
plt.show()

In [ ]:
plt.plot(history.history['mape'])
plt.plot(history.history['val_mape'])
plt.title('model mean absolute percentage error')
plt.ylabel('mape')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper right')
plt.show()

### insights
This model learns quite well 20 epochs in

## Evaluating model on new (unseen data)
should be the %age off of predicting the horizon price

In [ ]:
# Make sure best model is loaded and evaluate
model_6 = tf.keras.models.load_model("model_experiments/model_dense_multivariate")
loss = model_6.evaluate(X_test, y_test)

In [ ]:
# percentage
y_pred = model_6.predict(X_test)

In [ ]:
type(y_pred)

In [ ]:
y_pred

In [ ]:
y_test = np.array(y_test)

In [ ]:
type(y_test)

In [ ]:
y_test

In [ ]:
plt.plot(y_pred[:60], label="predicted log return")
plt.plot(y_test[:60], label="actual log return")
plt.title('forecasting future log returns')
plt.ylabel('price ($)')
plt.xlabel('time')
plt.legend(loc='upper right')
plt.show()

In [ ]:
plt.plot(y_pred, label="predicted log return")
plt.plot(y_test, label="actual log return")
plt.title('forecasting future log returns')
plt.ylabel('price ($)')
plt.xlabel('time')
plt.legend(loc='upper right')
plt.show()

In [ ]:
np.arange(10)

In [ ]:
plt.scatter(x=np.arange(60), y=y_pred[:60], label="predicted log return",)
plt.scatter(x=np.arange(60), y=y_test[:60], label="actual log return")
plt.plot(np.zeros(10))
plt.title('forecasting future log returns')
plt.ylabel('price ($)')
plt.xlabel('time')
plt.legend(loc='upper right')
plt.show()

In [ ]:
plt.plot(y_pred[:10], label="predicted log return",)
plt.plot(y_test[:10], label="actual log return")
plt.title('forecasting future log returns')
plt.ylabel('price ($)')
plt.xlabel('time')
plt.legend(loc='upper right')
plt.show()

## Go from log returns back to price

initial price should be price at X_test.timestamp-1

In [ ]:
p_0 = X_test

In [ ]:
# 1 step back as p_initial
p_0_time = X_test.index[0] - pd.Timedelta(X_test.index[0].freq)

In [ ]:
p_0_test = df.loc[p_0_time]['Close']

### The price should be initial_price * exp(log_ret.cumsum())

In [ ]:
tot_return_pct = y_pred.cumsum()
tot_return_pct

In [ ]:
y_pred_prices = p_0_test*np.exp(y_pred.cumsum())

In [ ]:
y_actual_prices = p_0_test*np.exp(y_test.cumsum())

In [ ]:
len(y_pred_prices), len(y_actual_prices)

In [ ]:
y_pred_prices[:500]

In [ ]:
y_actual_prices

In [ ]:
df['Close'][-10:]

In [ ]:
plt.plot(y_pred_prices[:60], label="predicted prices from log_returns")
plt.plot(y_actual_prices[:60], label="actual prices from log_returns")
plt.title('forecasting future price via log_returns')
plt.ylabel('price ($)')
plt.xlabel('time (s)')
plt.legend(loc='upper right')
plt.show()

In [ ]:
plt.plot(y_pred_prices[:500], label="predicted prices from log_returns")
plt.plot(y_actual_prices[:500], label="actual prices from log_returns")
plt.title('forecasting future price via log_returns')
plt.ylabel('price ($)')
plt.xlabel('time (s)')
plt.legend(loc='upper right')
plt.show()

### Observation: is okay for the first bit of data beyond train/val set, approx 50seconds, afterwards it flies!

In [ ]:
# for example, on the whole testing set
plt.plot(y_pred_prices, label="predicted prices from log_returns")
plt.plot(y_actual_prices, label="actual prices from log_returns")
plt.title('forecasting future price via log_returns')
plt.ylabel('price ($)')
plt.xlabel('time (s)')
plt.legend(loc='upper right')
plt.show()